# MPS canonical forms 

In [1]:

import numpy as np
from numpy import linalg as LA
from tensornetwork import ncon
import logging

import math 
import copy 

from myMPSstuff import myMPS, randMPS, bringCan, checkIdMatrix, expValOneSite


In [2]:
test = myMPS()

mpstest = np.random.rand(4,7,3,7)

test = myMPS(mpstest)



MPS with length 7 and physical d=2
chi [5, 5, 5, 5, 5, 5]
MPS with length 4 and physical d=3
chi [7, 7, 7]


## Are the SVs the same in the Left and Right forms ?

In [3]:
mymps2 = myMPS(randMPS(20))

bringCan(mymps2,mode='LR',epsTrunc=1e-8)
aaa = mymps2.SV

bringCan(mymps2,mode='L')
bbb = mymps2.SV

diff = [aaa[i]-bbb[i] for i in range(0,len(aaa)) ]

#[ aa[np.abs(aa) < 1e-14] = 0  for aa in diff]

#[ [el if el > 1e-14 else 0 for el in aa]  for aa in diff]

print("######")
#print([ [el if el > 1e-14 else 0 for el in aa]  for aa in diff])
print( np.all([ [aa < 1e-14 for aa in el] for el in diff]) )
print("######")


# looks like YES 

MPS with length 20 and physical d=2
chi [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
######
[True, True]
######


/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


## Check that we're actually in proper L-R canon forms by contracting a bit and see if we recover identity matrices and normalizations 

In [4]:
mymps3 = myMPS(randMPS(20))
aaa = bringCan(mymps3,mode='LR',epsTrunc=1e-6)
bbb = bringCan(mymps3,mode='L')

# So now the MPS should be normalized and in  LEFT canonical form 
# Check by contracting some A's 

mp = mymps3.MPS
sv = mymps3.SV
svinv = [svv**(-1) for svv in sv ]

m0m0 = ncon([mp[0],np.conjugate(mp[0])],[[1,-1],[1,-2]])
checkIdMatrix(m0m0)


m1m1 = ncon([mp[1],np.conjugate(mp[1])],[[1,2,-1],[1,2,-2]])
checkIdMatrix(m1m1)

m4m4 = ncon([mp[5],np.conjugate(mp[5])],[[1,2,-1],[1,2,-2]])
checkIdMatrix(m4m4)


# Looks good 

# Now try and build the B's using the recipe 

#Bj = (L_{j-1})^-1 @ Aj @ Lj

#print(np.shape(np.diag(svinv[2])),np.shape(mp[3]),np.shape(np.diag(sv[3])))


bb3 = ncon([np.diag(svinv[2]),  mp[3] ,  np.diag(sv[3]) ], [[-1,1],[1,-2,2],[2,-3]])

b3b3 = ncon([bb3,np.conjugate(bb3)],[[-1,1,2],[-2,1,2]])

checkIdMatrix(b3b3)

bb1 = ncon([np.diag(svinv[0]),  mp[1] ,  np.diag(sv[1]) ], [[-1,1],[1,-2,2],[2,-3]])

b1b1 = ncon([bb1,np.conjugate(bb1)],[[-1,1,2],[-2,1,2]])
checkIdMatrix(b1b1)

# Also looks good. 

# Work out the corners, we should have 

b0 = mp[0] @ np.diag(sv[0])
b0b0close =  ncon([b0,np.conjugate(b0)],[[1,2],[1,2]])
print(np.real_if_close(b0b0close))


MPS with length 20 and physical d=2
chi [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
identity, size = 2
identity, size = 4
identity, size = 5
identity, size = 5
identity, size = 2
1.0000000000000002


## Try computing an expectation value, first the "hard" way (ncon the whole thing), then using the right-canonical expression

In [5]:


testL = 10
testd = 2
testChi = 5

mymps4 = myMPS(randMPS(testL,chi=testChi, d=testd))

indicesMPS = [0]*2*testL

# Try to label p0 = 0, p1 = 1.. pL = L
# and l1 = L+1, l2=L+2, .. lL = 2L 

def pjj(jj: int):
    return jj+1

def ljj(jj: int):
    return jj+testL+1

def lbjj(jj: int):
    return jj+2*testL+1

tMPS = [mymps4.MPS[0]]
tMPS.append(np.conj(mymps4.MPS[0]))

iMPS = [[pjj(0),ljj(0)]]
iMPS.append([pjj(0),lbjj(0)])

for (jj, elem) in enumerate(mymps4.MPS[1:] ):

    #print(jj)
    tMPS.append(elem)
    tMPS.append(np.conj(elem))

    iMPS.append([ljj(jj),pjj(jj+1),ljj(jj+1)])
    iMPS.append([lbjj(jj),pjj(jj+1),lbjj(jj+1)])


iMPS[-2] = ([ljj(testL-2),pjj(testL-1)])
iMPS[-1] = ([lbjj(testL-2),pjj(testL-1)])

print(len(tMPS),len(iMPS))
print(iMPS)

tnc = ncon(tMPS,iMPS)

norm= tnc/np.real(tnc)

print(np.real_if_close(norm))






MPS with length 10 and physical d=2
chi [5, 5, 5, 5, 5, 5, 5, 5, 5]
20 20
[[1, 11], [1, 21], [11, 2, 12], [21, 2, 22], [12, 3, 13], [22, 3, 23], [13, 4, 14], [23, 4, 24], [14, 5, 15], [24, 5, 25], [15, 6, 16], [25, 6, 26], [16, 7, 17], [26, 7, 27], [17, 8, 18], [27, 8, 28], [18, 9, 19], [28, 9, 29], [19, 10], [29, 10]]
1.0


In [6]:
# Try to do it a bit better, define separately psi and psi*
# and compute the norm 

testL = 7
testd = 2
testChi = 5

mypsi = myMPS(randMPS(testL,chi=testChi, d=testd), offIndices=3)
mypsic = myMPS(randMPS(testL,chi=testChi, d=testd), offIndices=5)

conjMPS = [np.conj(m) for m in mypsi.MPS]
mypsic.MPS = conjMPS

"""
self.MPS = mMPS  
self.indices = indices
updateindices()
"""

print(mypsi.indices)
print(mypsi.getIndices())
print(mypsic.getIndices())

mtocon = copy.deepcopy(mypsi.MPS)
mtocon.extend(mypsic.MPS)

itocon = mypsi.getIndices()
itocon.extend(mypsic.getIndices())

print(itocon)

norm = ncon(mtocon,itocon)

normalized = norm/np.real(norm)

print(f"norm/alized = {np.real(norm)},{np.real_if_close(normalized)}")


# Now define the operator to exp-value

# Say we wanna compute <sigma_x> 
op1 = np.array([[0,1.],[1.,0]])

# at site r = 3 

psiOpens = [1,2,66,4,5,6,7]
psicOpens = [1,2,67,4,5,6,7]
op1Opens = [[66,67]]

mypsi.updateIndices(psiOpens)
mypsic.updateIndices(psicOpens)

mtocon = copy.deepcopy(mypsi.MPS)
mtocon.extend(mypsic.MPS)
mtocon.extend([op1])

print(len(mtocon),len(mypsi.MPS),len(mypsic.MPS),len(op1))

itocon = mypsi.getIndices()
itocon.extend(mypsic.getIndices())
itocon.extend(op1Opens)

#print(itocon)
#print(mtocon)
#print(len(itocon),len(mtocon))

tnc = ncon(mtocon,itocon)

print(np.real_if_close(tnc/norm))


# Now compute it with the canonical form 

copymypsi = copy.deepcopy(mypsi)
psiR = bringCan(copymypsi,mode='LR',epsTrunc=1e-12)


conTen = [np.diag(copymypsi.SV[1]),np.diag(copymypsi.SV[1]),copymypsi.MPS[2],np.conj(copymypsi.MPS[2]),op1]
conIdx = [[1,2],[1,3],[3,4,5],[2,6,5],[4,6]]

[print(np.shape(elem)) for elem in conTen]
print(len(conTen),len(conIdx))

print(np.real_if_close(ncon(conTen,conIdx)))

copymypsi2 = copy.deepcopy(mypsi)

print(expValOneSite(copymypsi2,op1,2))

MPS with length 7 and physical d=2
chi [5, 5, 5, 5, 5, 5]
MPS with length 7 and physical d=2
chi [5, 5, 5, 5, 5, 5]
[{'vL': 0, 'ph': 1, 'vR': 22}, {'vL': 22, 'ph': 2, 'vR': 23}, {'vL': 23, 'ph': 3, 'vR': 24}, {'vL': 24, 'ph': 4, 'vR': 25}, {'vL': 25, 'ph': 5, 'vR': 26}, {'vL': 26, 'ph': 6, 'vR': 27}, {'vL': 27, 'ph': 7, 'vR': 0}]
[[1, 22], [22, 2, 23], [23, 3, 24], [24, 4, 25], [25, 5, 26], [26, 6, 27], [27, 7]]
[[1, 36], [36, 2, 37], [37, 3, 38], [38, 4, 39], [39, 5, 40], [40, 6, 41], [41, 7]]
[[1, 22], [22, 2, 23], [23, 3, 24], [24, 4, 25], [25, 5, 26], [26, 6, 27], [27, 7], [1, 36], [36, 2, 37], [37, 3, 38], [38, 4, 39], [39, 5, 40], [40, 6, 41], [41, 7]]
norm/alized = 164429410.63905928,1.0
15 7 7 2
0.9761630417692106
Chis = [2, 4, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 4, 2]
(4, 4)
(4, 4)
(4, 2, 5)
(4, 2, 5)
(2, 2)
5 5
0.9761630417692098
Chis = [2, 4, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 4, 2]
0.9761630417692098


# The canonical form Gamma-Lambda 

Let's build it from our LR sweep
and check that we get the same results for our MPS 

In [7]:
mympsCC = myMPS(randMPS(20))

bringCan(mympsCC,mode='LR',epsTrunc=1e-8)

matsB = mympsCC.MPS[:]
svs = mympsCC.SV[:]

bringCan(mympsCC,mode='C',epsTrunc=1e-8)


matsG = mympsCC.MPS[:]
svinvs = mympsCC.SVinv[:]


# This brings it to the Gamma1-Gamma2-Gamma3... form, and separately we hold the SVs 

# Check that we can recover the L-R forms from it by contracting again everything on the left and the right
# of some middle site 


a0 =  matsB[19] 

checkIdMatrix( ncon([a0,np.conj(a0)],[[1,-1],[1,-2]]))


aa0 =  matsG[19] 

checkIdMatrix( ncon([aa0,np.conj(aa0)],[[1,-1],[1,-2]]))

aa0r = np.diag(svs[18]) @ matsG[19] 


checkIdMatrix( ncon([aa0r,np.conj(aa0r)],[[1,-1],[1,-2]]) )

MPS with length 20 and physical d=2
chi [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
Chis = [2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2]
identity, size = 2
Square but not id, difference = [[613.24464682 697.7499909 ]
 [697.7499909  793.90183399]]
identity, size = 2


True

In [8]:

# Now introduce MPOs 
